<a href="https://colab.research.google.com/github/trefftzc/cis677/blob/main/Exploring_julia_and_mpi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing Julia

In [1]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.11.0" # any version ≥ 0.7.0
JULIA_PACKAGES="MPI Statistics"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

# A first program with Julia and MPI

The following examples are taken from this web site:

https://juliaparallel.org/MPI.jl/stable/examples/

In [2]:
%%writefile 01-hello.jl
using MPI
MPI.Init()

comm = MPI.COMM_WORLD
println("Hello world, I am $(MPI.Comm_rank(comm)) of $(MPI.Comm_size(comm))")
MPI.Barrier(comm)
MPI.Finalize()

Overwriting 01-hello.jl


In [3]:
!ls

01-hello.jl  02-broadcast.jl  03-reduce.jl  04-sendrecv.jl  06-scatterv.jl  sample_data


# Install mpiexecjl to avoid problems with the version of MPI

In [5]:
!julia -e 'using MPI; MPI.install_mpiexecjl()'


ERROR: file `/root/.julia/bin/mpiexecjl` already exists; use `MPI.install_mpiexecjl(force=true)` to overwrite.
Stacktrace:
 [1] error(s::String)
   @ Base ./error.jl:35
 [2] install_mpiexecjl(; command::String, destdir::String, force::Bool, verbose::Bool)
   @ MPI ~/.julia/packages/MPI/TKXAj/src/mpiexec_wrapper.jl:18
 [3] install_mpiexecjl()
   @ MPI ~/.julia/packages/MPI/TKXAj/src/mpiexec_wrapper.jl:11
 [4] top-level scope
   @ none:1


In [4]:
%%shell
/root/.julia/bin/mpiexecjl --project=/content -n 3 julia 01-hello.jl

Hello world, I am 1 of 3
Hello world, I am 2 of 3
Hello world, I am 0 of 3


# An example of Broadcast

In [6]:
%%writefile 02-broadcast.jl
import MPI
MPI.Init()

comm = MPI.COMM_WORLD
N = 5
root = 0

if MPI.Comm_rank(comm) == root
    print(" Running on $(MPI.Comm_size(comm)) processes\n")
end
MPI.Barrier(comm)

if MPI.Comm_rank(comm) == root
    A = [i*(1.0 + im*2.0) for i = 1:N]
else
    A = Array{ComplexF64}(undef, N)
end

MPI.Bcast!(A, root, comm)

print("rank = $(MPI.Comm_rank(comm)), A = $A\n")

if MPI.Comm_rank(comm) == root
    B = Dict("foo" => "bar")
else
    B = nothing
end

B = MPI.bcast(B, root, comm)
print("rank = $(MPI.Comm_rank(comm)), B = $B\n")

if MPI.Comm_rank(comm) == root
    f = x -> x^2 + 2x - 1
else
    f = nothing
end

f = MPI.bcast(f, root, comm)
print("rank = $(MPI.Comm_rank(comm)), f(3) = $(f(3))\n")


Overwriting 02-broadcast.jl


Now executing the code

In [7]:
!/root/.julia/bin/mpiexecjl --project=/content -n 4 julia 02-broadcast.jl

 Running on 4 processes
rank = 0, A = ComplexF64[1.0 + 2.0im, 2.0 + 4.0im, 3.0 + 6.0im, 4.0 + 8.0im, 5.0 + 10.0im]
rank = 3, A = ComplexF64[1.0 + 2.0im, 2.0 + 4.0im, 3.0 + 6.0im, 4.0 + 8.0im, 5.0 + 10.0im]
rank = 1, A = ComplexF64[1.0 + 2.0im, 2.0 + 4.0im, 3.0 + 6.0im, 4.0 + 8.0im, 5.0 + 10.0im]
rank = 2, A = ComplexF64[1.0 + 2.0im, 2.0 + 4.0im, 3.0 + 6.0im, 4.0 + 8.0im, 5.0 + 10.0im]
rank = 0, B = Dict("foo" => "bar")
rank = 2, B = Dict("foo" => "bar")
rank = 1, B = Dict("foo" => "bar")
rank = 3, B = Dict("foo" => "bar")
rank = 0, f(3) = 14
rank = 3, f(3) = 14
rank = 1, f(3) = 14
rank = 2, f(3) = 14


Now an example of reduce

In [8]:
%%writefile 03-reduce.jl
# This example shows how to use custom datatypes and reduction operators
# It computes the variance in parallel in a numerically stable way

using MPI, Statistics

MPI.Init()
const comm = MPI.COMM_WORLD
const root = 0

# Define a custom struct
# This contains the summary statistics (mean, variance, length) of a vector
struct SummaryStat
    mean::Float64
    var::Float64
    n::Float64
end
function SummaryStat(X::AbstractArray)
    m = mean(X)
    v = varm(X,m, corrected=false)
    n = length(X)
    SummaryStat(m,v,n)
end

# Define a custom reduction operator
# this computes the pooled mean, pooled variance and total length
function pool(S1::SummaryStat, S2::SummaryStat)
    n = S1.n + S2.n
    m = (S1.mean*S1.n + S2.mean*S2.n) / n
    v = (S1.n * (S1.var + S1.mean * (S1.mean-m)) +
         S2.n * (S2.var + S2.mean * (S2.mean-m)))/n
    SummaryStat(m,v,n)
end

# Register the custom reduction operator.  This is necessary only on platforms
# where Julia doesn't support closures as cfunctions (e.g. ARM), but can be used
# on all platforms for consistency.
MPI.@RegisterOp(pool, SummaryStat)

X = randn(10,3) .* [1,3,7]'

# Perform a scalar reduction
summ = MPI.Reduce(SummaryStat(X), pool, comm; root)

if MPI.Comm_rank(comm) == root
    @show summ.var
end

# Perform a vector reduction:
# the reduction operator is applied elementwise
col_summ = MPI.Reduce(mapslices(SummaryStat,X,dims=1), pool, comm; root)

if MPI.Comm_rank(comm) == root
    col_var = map(summ -> summ.var, col_summ)
    @show col_var
end

Overwriting 03-reduce.jl


Now execute the reduction example

In [9]:
!/root/.julia/bin/mpiexecjl --project=/content -n 4 julia 03-reduce.jl

summ.var = 18.885008706354128
col_var = [0.9622197336623035 8.45653007750693 45.222046446282]


An example of send and receive

In [10]:
%%writefile 04-sendrecv.jl
using MPI
MPI.Init()

comm = MPI.COMM_WORLD
rank = MPI.Comm_rank(comm)
size = MPI.Comm_size(comm)

dst = mod(rank+1, size)
src = mod(rank-1, size)

N = 4

send_mesg = Array{Float64}(undef, N)
recv_mesg = Array{Float64}(undef, N)

fill!(send_mesg, Float64(rank))

rreq = MPI.Irecv!(recv_mesg, comm; source=src, tag=src+32)

print("$rank: Sending   $rank -> $dst = $send_mesg\n")
sreq = MPI.Isend(send_mesg, comm; dest=dst, tag=rank+32)

stats = MPI.Waitall([rreq, sreq])

print("$rank: Received $src -> $rank = $recv_mesg\n")

MPI.Barrier(comm)

Overwriting 04-sendrecv.jl


And now its execution:

In [11]:
!/root/.julia/bin/mpiexecjl --project=/content -n 4 julia 04-sendrecv.jl

1: Sending   1 -> 2 = [1.0, 1.0, 1.0, 1.0]
3: Sending   3 -> 0 = [3.0, 3.0, 3.0, 3.0]
0: Sending   0 -> 1 = [0.0, 0.0, 0.0, 0.0]
2: Sending   2 -> 3 = [2.0, 2.0, 2.0, 2.0]
1: Received 0 -> 1 = [0.0, 0.0, 0.0, 0.0]
0: Received 3 -> 0 = [3.0, 3.0, 3.0, 3.0]
3: Received 2 -> 3 = [2.0, 2.0, 2.0, 2.0]
2: Received 1 -> 2 = [1.0, 1.0, 1.0, 1.0]


An example of how to use scatter and gather

In [12]:
%%writefile 06-scatterv.jl
# This example shows how to use MPI.Scatterv! and MPI.Gatherv!
# roughly based on the example from
# https://stackoverflow.com/a/36082684/392585

using MPI

"""
    split_count(N::Integer, n::Integer)

Return a vector of `n` integers which are approximately equally sized and sum to `N`.
"""
function split_count(N::Integer, n::Integer)
    q,r = divrem(N, n)
    return [i <= r ? q+1 : q for i = 1:n]
end


MPI.Init()

comm = MPI.COMM_WORLD
rank = MPI.Comm_rank(comm)
comm_size = MPI.Comm_size(comm)

root = 0

if rank == root
    M, N = 4, 7

    test = Float64[i for i = 1:M, j = 1:N]
    output = similar(test)

    # Julia arrays are stored in column-major order, so we need to split along the last dimension
    # dimension
    M_counts = [M for i = 1:comm_size]
    N_counts = split_count(N, comm_size)

    # store sizes in 2 * comm_size Array
    sizes = vcat(M_counts', N_counts')
    size_ubuf = UBuffer(sizes, 2)

    # store number of values to send to each rank in comm_size length Vector
    counts = vec(prod(sizes, dims=1))

    test_vbuf = VBuffer(test, counts) # VBuffer for scatter
    output_vbuf = VBuffer(output, counts) # VBuffer for gather
else
    # these variables can be set to `nothing` on non-root processes
    size_ubuf = UBuffer(nothing)
    output_vbuf = test_vbuf = VBuffer(nothing)
end

if rank == root
    println("Original matrix")
    println("================")
    @show test sizes counts
    println()
    println("Each rank")
    println("================")
end
MPI.Barrier(comm)

local_size = MPI.Scatter(size_ubuf, NTuple{2,Int}, root, comm)
local_test = MPI.Scatterv!(test_vbuf, zeros(Float64, local_size), root, comm)

for i = 0:comm_size-1
    if rank == i
        @show rank local_test
    end
    MPI.Barrier(comm)
end

MPI.Gatherv!(local_test, output_vbuf, root, comm)

if rank == root
    println()
    println("Final matrix")
    println("================")
    @show output
end

Overwriting 06-scatterv.jl


In [13]:
!/root/.julia/bin/mpiexecjl --project=/content -n 4 julia 06-scatterv.jl

Original matrix
test = [1.0 1.0 1.0 1.0 1.0 1.0 1.0; 2.0 2.0 2.0 2.0 2.0 2.0 2.0; 3.0 3.0 3.0 3.0 3.0 3.0 3.0; 4.0 4.0 4.0 4.0 4.0 4.0 4.0]
sizes = [4 4 4 4; 2 2 2 1]
counts = [8, 8, 8, 4]

Each rank
rank = 0
local_test = [1.0 1.0; 2.0 2.0; 3.0 3.0; 4.0 4.0]
rank = 1
local_test = [1.0 1.0; 2.0 2.0; 3.0 3.0; 4.0 4.0]
rank = 2
local_test = [1.0 1.0; 2.0 2.0; 3.0 3.0; 4.0 4.0]
rank = 3
local_test = [1.0; 2.0; 3.0; 4.0;;]

Final matrix
output = [1.0 1.0 1.0 1.0 1.0 1.0 1.0; 2.0 2.0 2.0 2.0 2.0 2.0 2.0; 3.0 3.0 3.0 3.0 3.0 3.0 3.0; 4.0 4.0 4.0 4.0 4.0 4.0 4.0]
